## **Path-following**

Based on the code from Tutorial 2 that captures images from the camera. The code that's been added is contained between comments like:<br>
------------------------X------------------------<br>
&nbsp;&nbsp;&nbsp;&nbsp;new code<br>
------------------------^------------------------<br>
<br>
Where X matches X) in the code explanation below:<br>
1) Initialises robot and starts movement loop.<br>
2) Get image, take a horizontal line across the centre of the image 10 pixels tall (self.middle\_line), split into centre, left, and right sections (self.middle\_line\_centre, ...\_left, ...\_right, respectively), and feed each into a function that detects the presence of blue or yellow pixels(.has\_blue\_or\_yellow()) to set movement values (self.forward, self.left, and self.right).<br>
3) The function that detects the presence of yellow or blue.<br>
4) Same as 2) but called as part of the threaded process (2 is only called with the \_\_init\_\_).<br>

**Step 1.**

Run the following code to define the RGBD sensor class.

In [1]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    color_value = traitlets.Any()
    
#                                     ------------------------3------------------------
    @staticmethod
    def has_blue_or_yellow(slice_array):
        slice_array = slice_array.astype(np.int16)
        # Function that returns True if the array contains a yellow or blue pixel, o/w False
#         print(slice_array.shape)
        blue = slice_array[:,0]
#         print("blue {}".format(blue))
        red = slice_array[:,2]
#         print("red {}".format(red))
        green = slice_array[:,1]
        blue_red_diffs = blue - red
#         print("blue-red diffs {}".format(blue_red_diffs))
        red_blue_diffs = red - blue
#         print("red-blue diffs {}".format(red_blue_diffs))
        red_green_diffs = red - green
        blue_pixels = np.where(blue_red_diffs >= 70)
#         print("blue pix {} | {}".format(blue_pixels, len(blue_pixels[0])))
        yellow_pixels = np.where((red_blue_diffs >= 70) & (red_green_diffs <= 70))
#         print("yellow pix {} | {}".format(yellow_pixels, len(yellow_pixels[0])))
        blue_or_yellow_pixels = len(blue_pixels[0]) + len(yellow_pixels[0])
#         print("blue or yellow pix: {}".format(blue_or_yellow_pixels))
        if (blue_or_yellow_pixels > 20):
            return True
        else:
            return False
#                                     ------------------------^------------------------

    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()
        
        #constants for line
        self.MIDDLE_LINE_WIDTH = 10
        self.MIDDLE_LINE_POS = 240
        self.middle_line_bottom = int(self.MIDDLE_LINE_POS - self.MIDDLE_LINE_WIDTH / 2)
        self.middle_line_top = int(self.MIDDLE_LINE_POS + self.MIDDLE_LINE_WIDTH / 2)
        self.SIDE_LINE_WIDTH = 10
        self.LEFT_LINE_CENTRE = 160
        self.left_line_start = int(self.LEFT_LINE_CENTRE - self.SIDE_LINE_WIDTH / 2)
        self.left_line_end = int(self.LEFT_LINE_CENTRE + self.SIDE_LINE_WIDTH / 2)
        self.RIGHT_LINE_CENTRE = 480
        self.right_line_start = int(self.RIGHT_LINE_CENTRE - self.SIDE_LINE_WIDTH / 2)
        self.right_line_end = int(self.RIGHT_LINE_CENTRE + self.SIDE_LINE_WIDTH / 2)
#         === CORNER FIX ===
#         self.TOPLINE_POS = 120
#         self.top_line_bottom = int(self.TOP_LINE_POS - self.MIDDLE_LINE_WIDTH / 2)
#         self.top_line_top = int(self.TOP_LINE_POS + self.MIDDLE_LINE_WIDTH / 2)
#         ================== 
        
        self.MIDDLE_LINE_CENTRE_WIDTH =180
        self.middle_line_start = int((self.color_width / 2) - (self.MIDDLE_LINE_CENTRE_WIDTH / 2))
        self.middle_line_end = int((self.color_width / 2) + (self.MIDDLE_LINE_CENTRE_WIDTH / 2))

#                                     ------------------------2------------------------
        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
#         image[235:245,329:350] = 0
        # Select horizontal line of pixels across the centre of the image of height 10 and vertical
        # lines across the left and right of the image of widths 10.
        self.middle_line = image[self.middle_line_bottom:self.middle_line_top,:]
        self.left_line = image[self.middle_line_top:,self.left_line_start:self.left_line_end]
        self.left_line = np.reshape(self.left_line,((self.color_height-self.middle_line_top)*self.SIDE_LINE_WIDTH,3))
        self.right_line = image[self.middle_line_top:,self.right_line_start:self.right_line_end]
        self.right_line = np.reshape(self.right_line,((self.color_height-self.middle_line_top)*self.SIDE_LINE_WIDTH,3))
#         === CORNER FIX ===
#         self.top_line = image[self.top_line_bottom:self.top_line_top,:]
#         self.top_line_left = self.top_line[:,:self.middle_line_start]
#         self.top_line_left = np.reshape(self.top_line_left,(self.top_line_start*self.MIDDLE_LINE_WIDTH,3))
#         self.top_line_right = self.top_line[:,self.middle_line_end:]
#         self.top_line_right = np.reshape(self.top_line_right,(self.top_line_start*self.MIDDLE_LINE_WIDTH,3))
#         ==================         
        # Split middle line into centre, left, and right sub-arrays, reshape into 2D,
        # and pass into has_blue_or_yellow to decide movement.
        self.middle_line_centre = self.middle_line[:,self.middle_line_start:self.middle_line_end]
        self.middle_line_centre = np.reshape(self.middle_line_centre,(self.MIDDLE_LINE_CENTRE_WIDTH*self.MIDDLE_LINE_WIDTH,3))
        self.forward = self.has_blue_or_yellow(self.middle_line_centre)
        self.middle_line_left = self.middle_line[:,:self.middle_line_start]
        self.middle_line_left = np.reshape(self.middle_line_left,(self.middle_line_start*self.MIDDLE_LINE_WIDTH,3))
        self.forward_left = self.has_blue_or_yellow(self.middle_line_left) # and self.has_blue_or_yellow(self.top_line_left)
        self.middle_line_right = self.middle_line[:,self.middle_line_end:]
        self.middle_line_right = np.reshape(self.middle_line_right,(self.middle_line_start*self.MIDDLE_LINE_WIDTH,3))
        self.forward_right = self.has_blue_or_yellow(self.middle_line_right) # and self.has_blue_or_yellow(self.top_line_right)
        self.left = self.has_blue_or_yellow(self.left_line) # and self.has_blue_or_yellow(self.top_line_left)
        self.right = self.has_blue_or_yellow(self.right_line) # and self.has_blue_or_yellow(self.top_line_right)
        
#                                     ------------------------^------------------------
        self.color_value = image

        #start capture the first depth image
        depth_frame = frames.get_depth_frame()           
        depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        depth_colormap = cv2.putText(depth_colormap, "blue: " + str(image[246,351][0]) + " red: " + str(image[246,351][2]), (213,240), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        self.depth_value = depth_colormap   


    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
#                                     ------------------------4------------------------
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
#             image[235:245,290:350] = 0
            # Select horizontal line of pixels across the centre of the image of height 10 and vertical
            # lines across the left and right of the image of widths 10.
            self.middle_line = image[self.middle_line_bottom:self.middle_line_top,:]
            self.left_line = image[self.middle_line_top:,self.left_line_start:self.left_line_end]
            self.left_line = np.reshape(self.left_line,((self.color_height-self.middle_line_top)*self.SIDE_LINE_WIDTH,3))
            self.right_line = image[self.middle_line_top:,self.right_line_start:self.right_line_end]
            self.right_line = np.reshape(self.right_line,((self.color_height-self.middle_line_top)*self.SIDE_LINE_WIDTH,3))
#             === CORNER FIX ===
#             self.top_line = image[self.top_line_bottom:self.top_line_top,:]
#             self.top_line_left = self.top_line[:,:self.middle_line_start]
#             self.top_line_left = np.reshape(self.top_line_left,(self.top_line_start*self.MIDDLE_LINE_WIDTH,3))
#             self.top_line_right = self.top_line[:,self.middle_line_end:]
#             self.top_line_right = np.reshape(self.top_line_right,(self.top_line_start*self.MIDDLE_LINE_WIDTH,3))
#             ==================         
            # Split middle line into centre, left, and right sub-arrays, reshape into 2D,
            # and pass into has_blue_or_yellow to decide movement.
            self.middle_line_centre = self.middle_line[:,self.middle_line_start:self.middle_line_end]
            self.middle_line_centre = np.reshape(self.middle_line_centre,(self.MIDDLE_LINE_CENTRE_WIDTH*self.MIDDLE_LINE_WIDTH,3))
            self.forward = self.has_blue_or_yellow(self.middle_line_centre)
            self.middle_line_left = self.middle_line[:,:self.middle_line_start]
            self.middle_line_left = np.reshape(self.middle_line_left,(self.middle_line_start*self.MIDDLE_LINE_WIDTH,3))
            self.forward_left = self.has_blue_or_yellow(self.middle_line_left) # and self.has_blue_or_yellow(self.top_line_left)
            self.middle_line_right = self.middle_line[:,self.middle_line_end:]
            self.middle_line_right = np.reshape(self.middle_line_right,(self.middle_line_start*self.MIDDLE_LINE_WIDTH,3))
            self.forward_right = self.has_blue_or_yellow(self.middle_line_right) # and self.has_blue_or_yellow(self.top_line_right)
            self.left = self.has_blue_or_yellow(self.left_line) # and self.has_blue_or_yellow(self.top_line_left)
            self.right = self.has_blue_or_yellow(self.right_line) # and self.has_blue_or_yellow(self.top_line_right)
#                                     ------------------------^------------------------
            
            self.color_value = image #assign the numpy array image to the color_value variable

            depth_frame = frames.get_depth_frame() #get the depth image           
            depth_image = np.asanyarray(depth_frame.get_data()) #convert depth data to numpy array
            #conver depth data to BGR image for displaying purpose
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            depth_colormap = cv2.putText(depth_colormap, "blue: " + str(image[246,351][0]) + " red: " + str(image[246,351][2]), (213,240), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            self.depth_value = depth_colormap #assign the color BGR image to the depth value
    
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data


**Step 2.**

Create a widget to display the images in real-time. Monitor the image via the observe function

In [2]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
import sys

#create widgets for the displaying of the image
display_color = widgets.Image(format='jpeg', width='45%') #determine the width of the color image
display_depth = widgets.Image(format='jpeg', width='45%')  #determine the width of the depth image
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
display(sidebyside) #display the widget

#callback function, invoked when traitlets detects the changing of the color image
def process(change):
    
    image = change['new'] #retrieve data from the input dict
    if camera.forward:
        image[camera.middle_line_bottom:camera.middle_line_top,camera.middle_line_start:camera.middle_line_end] = [0, 255, 0]
    if camera.forward_left:
        image[camera.middle_line_bottom:camera.middle_line_top,:camera.middle_line_start] = [0, 122, 0]
    if camera.forward_right:
        image[camera.middle_line_bottom:camera.middle_line_top,camera.middle_line_end:] = [0, 122, 0]
    if camera.left:
        image[camera.middle_line_top:,camera.left_line_start:camera.left_line_end] = [0, 255, 0]
    if camera.right:
        image[camera.middle_line_top:,camera.right_line_start:camera.right_line_end] = [0, 255, 0]

    display_color.value = bgr8_to_jpeg(cv2.resize(image,(160,120)))
    display_depth.value = bgr8_to_jpeg(cv2.resize(camera.depth_value,(160,120)))

#processing({'new': camera.color_value})
#the camera.observe function will monitor the color_value variable. If this value changes, the processing function will be excuted.
camera.observe(process, names='color_value')

**Step 3.**

Start a while loop that reads the movement value from the Camera class (bad separation of concerns, I know) and moves the robot accordingly. 

In [5]:
#                                     ------------------------1------------------------
import time
from RobotClass import Robot
robot = Robot()
start_time = time.time()

FORWARD_STRENGTH = 0.3
TURN_STRENGTH = 0.5
TURN_TIME = 0.4
robot.forward(FORWARD_STRENGTH)
# Run for two minutes
while time.time() - start_time < 120:
    if (camera.forward):
        robot.forward(FORWARD_STRENGTH)
        print("forward")
    elif (camera.forward_left):
        robot.forward_left(speed=FORWARD_STRENGTH * 3)
#         robot.left(speed=0.7)
        print("forward left")
    elif (camera.forward_right):
        robot.forward_right(speed=FORWARD_STRENGTH * 3)
#         robot.right(speed=0.7)
        print("forward right")
    elif (camera.left):
        robot.left(speed=TURN_STRENGTH)
        time.sleep(TURN_TIME)
#         robot.forward(FORWARD_STRENGTH)
        print("left")
    elif (camera.right):
        robot.right(speed=TURN_STRENGTH)
        time.sleep(TURN_TIME)
#         robot.forward(FORWARD_STRENGTH)
        print("right")
    else:
#         robot.forward(FORWARD_STRENGTH)
#         print("panic forward")
        robot.backward(FORWARD_STRENGTH)
        print("panic backward")
robot.stop()
#                                     ------------------------^------------------------

forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward
forward


**Utilities**

Stop the observation and the data capturing thread.

In [ ]:
#stop observe the changing of the variables
camera.unobserve_all()
camera.stop() #stop data capturing thread
#you will find that the image stops changing in the above window

Restart the observation and the data capturing thread.

In [ ]:
#start data capturing thread again
camera.start()
camera.observe(process, names='color_value') #start monitor the changing of the color image 
#you will find that the image start changing again in the above window

In [4]:
robot.stop()